###JOURNAL SCRAPING PROJECT

###Installing Newspaper Article libraries to Scrap web pages.

In [0]:
!pip install newspaper3k

In [0]:
!apt-get install python3-bs4 

In [0]:
import nltk
nltk.download('punkt')

###Getting Data from Beautiful Soap Which has libraries to scrap data

In [0]:
import httplib2
import nltk
from bs4 import BeautifulSoup, SoupStrainer
import requests 
import csv 
from newspaper import Article 
from newspaper import fulltext
import pandas as pd
import numpy as np
import collections
articler,l = {},{}
#l ={}
cn = 0
http = httplib2.Http()
#calling URL  For Journal and Scrap each page of Journal year:
for i in range(1,52):
  t = 'https://gsejournal.biomedcentral.com/articles?tab=keyword&searchType=journalSearch&sort=PubDate&volume='
  r = i
  k,f= [],[]
  #f = []
  for j in range(0,9):
      s = f'&page={j}'
      status, response = http.request(f'{t}{r}{s}')
      for link in BeautifulSoup(response, 'html.parser', parseOnlyThese=SoupStrainer('a')):
        if link.has_attr('href'):
          k.append(link['href'])
  x = [item for item, count in collections.Counter(k).items() if count > 1]
  for p in x:
    if p.startswith('/articles/10.1186')==True:
      f.append(p)
      cn+=1
  articler[f'{i}'] = f
  l[f'{i}'] = 1968+i 
  year = dict((l[key], value) for (key, value) in articler.items())
  

##Defination(Function) for calling  a year .User can input year into function and it will give all 8 Feilds.

In [0]:
###Definatiion for scraping Data.
def volume(x):
  cn =0
  scrap = {}
  s1 = 'https://gsejournal.biomedcentral.com'
  for i in year.keys():
    if(i==x):
      for j in year[i]:
        m = 'https://'
        n = 'doi.org/'
        URL = f'{s1}{j}'
        r = requests.get(URL)
        article_name = Article(f'{s1}{j}', language="en")
        article_name.download()
        #print(f'{s1}{j}')
        soup = BeautifulSoup(r.content, 'html5lib') 
        table = soup.find('meta', attrs = {'name':'citation_abstract'})
        table1 = soup.find('meta', attrs = {'name':'prism.doi'})
        p = table1['content'][4:]
        article_name.parse()
        article_name.nlp()
        #print(article_name.publish_date)
        #print(article_name.keywords)
        #print(article_name.title) 
        #print("DOI",f'{m}{n}{p}',"\nArticle Name\n",article_name.title)
        auth = []
        for link in soup.find_all('meta',attrs = {'name':'citation_author'}):
          auth.append(link.get('content')) 
        #print("corresponding author",table4.string)
        aff= []
        for k in soup.find_all('span',itemprop='affiliation'):
          #print(k.find('meta')['content'])
          aff.append(k.find('meta',attrs={'itemprop':'address'})['content'])
          #print(k.find('meta',attrs={'itemprop':'address'})['content'])
        #print("email",f'{s1}{tb5}')
        full = requests.get(URL).text
        #print(fulltext(full))
        scrap.setdefault('DOI',[]).append(f'{m}{n}{p}')
        scrap.setdefault('Title',[]).append(article_name.title)
        scrap.setdefault('Authors',[]).append(auth)
        scrap.setdefault('Author Affiliations',[]).append(aff)
        scrap.setdefault('Publication Date',[]).append(article_name.publish_date)
        scrap.setdefault('Abstract',[]).append(table['content'])
        scrap.setdefault('keywords',[]).append(article_name.keywords)
        scrap.setdefault('full text',[]).append(fulltext(full))
        cn+=1
  return scrap

          

##Calling Function

In [0]:
t = volume(1969)

##Defination for making a Text for input  Year

In [0]:
def makingtext(year):
  for i in range(1,52):  
    x = 1968+i
    if (year = x):
      t = volume(x)
      df = pd.DataFrame(t)
      p = 'Genetic'
      j = f'{p}{x}'+'.txt'
      export_csv = df.to_csv (f'{j}', index = None, header=True) #Don't forget to add '.csv' at the end of the path
    return export_csv



###ReadingText file


In [0]:
files = pd.read_csv(makingtext(year),header=None)

In [0]:
files

###Making Directries for all Journal Year:to load HTML page

In [0]:
for i in range(1,52):
  x = 1968+i
  os.mkdir(f'{x}')

In [0]:
###Generating all Html page:
import urllib.request
for i in range(1,52):
  x = 1968+i
  t  = volume(x)
  df = pd.DataFrame(t)
  k = '/content/drive/My Drive/Rproject/Scrap/'
  m = f'{k}{x}'
  os.chdir(m)
  for i in range(len(df['DOI'])):
    try:
      htmlfile = urllib.request.urlopen(df['DOI'][i])
      htmltext = htmlfile.read().decode('utf-8')
      t =df['DOI'][i][16:23]+'-'+df['DOI'][i][24:]
      z = f'{t}'
      f  = open(z+'.html','w+')
      f.write(htmltext)
    except:
      pass


### NO coresponding Author For this Data Couple of Article have a correspond's Author after 2001 so defination for coresponding author and email.

In [0]:
      def corespondemce(x):
        cn =0
        scrap = {}
        s1 = 'https://gsejournal.biomedcentral.com'
        for i in year.keys():
          if(i==x):
            for j in year[i]:
              m = 'https://'
              n = 'doi.org/'
              URL = f'{s1}{j}'
              r = requests.get(URL)
              try:
                soup = BeautifulSoup(r.content, 'html5lib') 
                article_name = Article(f'{s1}{j}', language="en")
                article_name.download()
                table4 = soup.find('a',attrs={'id':'corresp-c1'})
                table5 = soup.find('a',attrs={'id':'corresp-c1'})
                tb5 = table5['href']
                scrap.setdefault('Corresponding Author',[]).append(table4.string)
                scrap.setdefault('Corresponding Author s Email',[]).append(f'{s1}{tb5}')
              except:
                print("There is no Corespondece for this year")
            return scrap


In [0]:
corespondemce(1969)

####THANK YOU#####
